
# Demonstration of the Hybrid Kernelised Expaction Maximisation (HKEM) reconstruction with SIRF
This demonstration shows how to use HKEM and investigate the role of each kernel parameter in edge preservation and noise suppression.


Authors: Daniel Deidda, Kris Thielemans and Evgueni Ovtchinnikov, Richard Brown  
First version: 30th of September 2019  
Second Version: 6th of November 2019

CCP PETMR Synergistic Image Reconstruction Framework (SIRF)  
Copyright 2019  National Physical Laboratory
Copyright 2019  Rutherford Appleton Laboratory STFC  
Copyright 2019  University College London.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

N.B.: You need to have run the [brainweb](./BrainWeb.ipynb) notebook first.

## Further reading!

Lastly, if you wish to read about HKEM, check out the original article by D. Deidda et al. (2019), available here: https://iopscience.iop.org/article/10.1088/1361-6420/ab013f.

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
import string
#import scipy
#from scipy import optimize
import sirf.STIR as pet
from sirf.Utilities import examples_data_path

# Get to correct directory
os.chdir(examples_data_path('Synergistic'))

#%% copy files to working folder and change directory to where the output files are
shutil.rmtree('working_folder/HKEM',True)
shutil.copytree('brainweb','working_folder/HKEM')
os.chdir('working_folder/HKEM')

In [ ]:
#%% some handy function definitions
def imshow_hot(image, limits, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image, cmap="hot")
    if len(limits)==0:
        limits = [image.min(), image.max()]

    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

def imshow(image, limits, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image)
    if len(limits)==0:
        limits = [image.min(), image.max()]

    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

def imshow_gray(image, limits, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image, cmap="gray")
    if len(limits)==0:
        limits = [image.min(), image.max()]

    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV"""
    filter = pet.TruncateToCylinderProcessor()
    filter.apply(image)

### Load some data and set some values

In [ ]:
full_sino = pet.AcquisitionData('FDG_tumour_sino_noisy.hs')

#%% the following multiplication is just in case the noise sinogram has a different scale from the noiseless
# you can remove it in a real data situation
sino = sino*1000

atten = pet.ImageData('uMap_small.hv')

# Anatomical image
anatomical = pet.ImageData('T2_small.hv')
anatomical.fill(make_positive(anatomical.as_array()))

#%%  create initial image
init_image=atten.get_uniform_copy(atten.as_array().max()*.1)
make_cylindrical_FOV(init_image)

In [ ]:
image = pet.ImageData('FDG_tumour.hv')
image_array = image.as_array()
#%% save max for future displays
cmax = image_array.max()*.6

In [ ]:
## Show anatomical image and true image
anatomical_array=anatomical.as_array()
atten_array=atten.as_array()
im_slice = 62 #atten_array.shape[0]//2

plt.figure()
plt.subplot(1,2,1)
imshow_gray(anatomical_array[im_slice,:,:,], [0,220],'Anatomical image')
plt.subplot(1,2,2)
imshow_hot(image_array[im_slice,:,:,], [0,cmax*2],'True image')

## Set up the acquisition model and objective function

In [ ]:
# Use SSRB to create smaller sinogram to speed up calculations if needed
# if you want to make things faster you can rebin your data by compressing axial and view bins
# this will affect the quality of the reconstructed images. See help(full_sino.rebin)
# if you have enough computational power you can try unsetting max_in_segment_num_to_process
sino = full_sino.rebin(1, num_views_to_combine=1,max_in_segment_num_to_process=0,do_normalisation=False)


In [ ]:
#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
am.set_num_tangential_LORs(5)

# Set up sensitivity due to attenuation
asm_attn = pet.AcquisitionSensitivityModel(atten, am)
asm_attn.set_up(sino)
bin_eff = pet.AcquisitionData(sino)
bin_eff.fill(1.0)
asm_attn.unnormalise(bin_eff)
asm_attn = pet.AcquisitionSensitivityModel(bin_eff)

# Set sensitivity of the model and set up
am.set_acquisition_sensitivity(asm_attn)
am.set_up(sino,atten)

#%% create objective function
obj_fun = pet.make_Poisson_loglikelihood(sino)
obj_fun.set_acquisition_model(am)

##  create KOSMAPOSL reconstructor
This implements the Ordered Subsets HKEM
In this section we define all parameter

In [ ]:
recon = pet.KOSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)

recon.set_input(sino)

recon.set_anatomical_prior(anatomical)
recon.set_num_non_zero_features(1)
recon.set_num_subsets(21)
recon.set_num_subiterations(42)

## Study parameter sigma_m (MR edge preservation) 


In [ ]:
#%% reconstruct the image 
H1m_reconstructed_image = [] 

#fix other parameters
recon.set_num_neighbours(3)
recon.set_sigma_p(0.2)
recon.set_sigma_dm(5.0)
recon.set_sigma_dp(5.0)

sigma_m={0.05, 0.2, 1}
ii=0
for i in sigma_m:

    H1m_reconstructed_image.append(init_image.clone())
    
    recon.set_sigma_m(i)

#   set up the reconstructor
    recon.set_hybrid(True)
    recon.set_up(H1m_reconstructed_image[ii])
    recon.reconstruct(H1m_reconstructed_image[ii])


    ii=ii+1;

In [ ]:
# %% bitmap display of images
# define lists
H1m_reconstructed_array = []
H1m_error_array = []

ii=0

for i in sigma_m:

    H1m_reconstructed_array.append(H1m_reconstructed_image[ii].as_array())
    H1m_error_array.append(image_array - H1m_reconstructed_array[ii])
    
  
    j="{}".format(i)
    plt.figure()
    plt.subplot(1,3,1)
    imshow_hot(image_array[im_slice,:,:,], [0,cmax*2],'True image')
    plt.subplot(1,3,2)
    imshow_hot(H1m_reconstructed_array[ii][im_slice,:,:,], [0,cmax*2], 'sigma_m='+j)
    plt.subplot(1,3,3)
    imshow(H1m_error_array[ii][im_slice,:,:,], [-cmax*0.5,cmax*0.5], 'HKEM error')

    ii=ii+1;

## Study parameter sigma_p (PET edge preservation)


In [ ]:
#%% reconstruct the image 
H1p_reconstructed_image = [] 

#fix other parameters
recon.set_num_neighbours(3)
recon.set_sigma_m(0.2)
recon.set_sigma_dm(5.0)
recon.set_sigma_dp(5.0)

sigma_p={0.05, 2}
ii=0
for i in sigma_p:

    H1p_reconstructed_image.append(init_image.clone())

    recon.set_sigma_p(i)
#   set up the reconstructor
    recon.set_hybrid(True)
    recon.set_up(H1p_reconstructed_image[ii])
    recon.reconstruct(H1p_reconstructed_image[ii])
    ii=ii+1;

In [ ]:
H1p_reconstructed_image.append(H1p_reconstructed_image[1])
#%% bitmap display of images
# define lists
H1p_reconstructed_array = []
H1p_error_array = []
ii=0
sigma_p={0.05, 2,0.2}
for i in sigma_p:

    j="{}".format(i)

    H1p_reconstructed_array.append(H1p_reconstructed_image[ii].as_array())

    H1p_error_array.append(image_array - H1p_reconstructed_array[ii])

    plt.figure()
    plt.subplot(1,3,1)
    imshow_hot(image_array[im_slice,:,:,], [0,cmax*2],'True image')
    plt.subplot(1,3,2)
    imshow_hot(H1p_reconstructed_array[ii][im_slice,:,:,], [0,cmax*2], 'sigma_p='+j)
    plt.subplot(1,3,3)
    imshow(H1p_error_array[ii][im_slice,:,:,], [-cmax*0.5,cmax*0.5], 'HKEM error')

    ii=ii+1;

## Study parameter sigma_d (smoothing, depends on the voxel size)


In [ ]:
#%% reconstruct the image 
H1d_reconstructed_image = [] 

#fix other parameters
recon.set_num_neighbours(3)
recon.set_sigma_m(0.2)
recon.set_sigma_p(0.2)

sigma_dm={0.5, 1}
ii=0
for i in sigma_dm:

    H1d_reconstructed_image.append(init_image.clone())

    recon.set_sigma_dp(i)
    recon.set_sigma_dm(i)

   #   set up the reconstructor
    recon.set_hybrid(True)
    recon.set_up(H1d_reconstructed_image[ii])
    recon.reconstruct(H1d_reconstructed_image[ii])

    ii=ii+1;

In [ ]:
H1d_reconstructed_image.append(H1m_reconstructed_image[1])
#%% bitmap display of images
# define lists
H1d_reconstructed_array = []
H1d_error_array = []
ii=0
sigma_dm={0.5, 1, 5}
for i in sigma_dm:

    j="{}".format(i)

    H1d_reconstructed_array.append(H1d_reconstructed_image[ii].as_array())

#   anatomical_image_array = anatomical_image.as_array()
    H1d_error_array.append(image_array - H1d_reconstructed_array[ii])

    plt.figure()
    plt.subplot(1,3,1)
    imshow_hot(image_array[im_slice,:,:,], [0,cmax*2],'True image')
    plt.subplot(1,3,2)
    imshow_hot(H1d_reconstructed_array[ii][im_slice,:,:,], [0,cmax*2], 'sigma_dm='+j)
    plt.subplot(1,3,3)
    imshow(H1d_error_array[ii][im_slice,:,:,], [-cmax*0.5,cmax*0.5], 'HKEM error')

    ii=ii+1
    

## Study parameter "neighbourhood size", n
try to rebin the data if it is too slow (sino.rebin(segments, views) )

In [ ]:
#%% reconstruct the image 
H1n_reconstructed_image = [] 

#fix other parameters
recon.set_sigma_m(0.2)
recon.set_sigma_p(0.2)
recon.set_sigma_dm(5.0)
recon.set_sigma_dp(5.0)

n={1, 5}
ii=0
for i in n:

    H1n_reconstructed_image.append(init_image.clone())

    recon.set_num_neighbours(i)

#   set up the reconstructor
    recon.set_hybrid(True)
    recon.set_up(H1n_reconstructed_image[ii])
    recon.reconstruct(H1n_reconstructed_image[ii])

    ii=ii+1;

In [ ]:
H1n_reconstructed_image.pop(1)
H1n_reconstructed_image.append(H1m_reconstructed_image[1])
#%% bitmap display of images
# define lists

H1n_reconstructed_array = []
H1n_error_array = []
ii=0
n={1, 5, 3}
for i in n:

    j="{}".format(i)
    
    H1n_reconstructed_array.append(H1n_reconstructed_image[ii].as_array())

#   anatomical_image_array = anatomical_image.as_array()
    H1n_error_array.append(image_array - H1n_reconstructed_array[ii])

    plt.figure()
    plt.subplot(1,3,1)
    imshow_hot(image_array[im_slice,:,:,], [0,cmax*2],'True image')
    plt.subplot(1,3,2)
    imshow_hot(H1n_reconstructed_array[ii][im_slice,:,:,], [0,cmax*2], 'HKEM, N='+j)
    plt.subplot(1,3,3)
    imshow(H1n_error_array[ii][im_slice,:,:,], [-cmax*0.5,cmax*0.5], 'HKEM error')

    ii=ii+1;

## Reconstruct KEM By setting hybrid to false



In [ ]:
#HKEM image is: H1n_reconstructed_array
    
    #%% reconstruct the image 
H0_reconstructed_image = [] 

#fix other parameters
recon.set_sigma_m(0.2)
recon.set_sigma_p(0.2)
recon.set_sigma_dm(5.0)
recon.set_sigma_dp(5.0)

H0_reconstructed_image.append(init_image.clone())
recon.set_num_neighbours(5)

#   set up the reconstructor
recon.set_hybrid(False)
recon.set_up(H0_reconstructed_image[0])
recon.reconstruct(H0_reconstructed_image[0])


## Compare HKEM and KEM 


In [ ]:
H0_reconstructed_array= H0n_reconstructed_image[0].as_array()
plt.figure()
plt.subplot(1,2,1)
imshow_hot(H0_reconstructed_array[im_slice,:,:,], [0,cmax*2.], 'KEM')
plt.subplot(1,2,2)
imshow_hot(H1n_reconstructed_array[2][im_slice,:,:,], [0,cmax*2.], 'HKEM')

## 1) what difference can you see when you change each parameter? and between HKEM and KEM?

## 2) Create misalignment between Anatomical image and PET image

In [1]:
## you can create misalignment by shifting or rotation the anatomical image like in the notebook "BrainWeb"


In [ ]:
## set KOSMAPOSL reconstructor using the new anatomical image and reconstruct as above. 

In [ ]:
## plot images: Describe what was the effect of this misalignment

## 3) Run a OSEM reconstruction and align the anatomical image from the previous exercise with the OSEM image.


In [ ]:
## use what you learned in the PET reconstruction notebooks

## 4) Similar exercise could be done with other type of anatomical information. You could have a look at the de_Pierro_MAPEM notebook and repeat these exrcises